In [1]:
import pandas as pd
import scipy.stats as st
import numpy as np

### Нахождение доверительного интервала

In [3]:
sample = pd.Series([
    439, 518, 452, 505, 493, 470, 498, 442, 497, 
    423, 524, 442, 459, 452, 463, 488, 497, 500,
    476, 501, 456, 425, 438, 435, 516, 453, 505, 
    441, 477, 469, 497, 502, 442, 449, 465, 429,
    442, 472, 466, 431, 490, 475, 447, 435, 482, 
    434, 525, 510, 494, 493, 495, 499, 455, 464,
    509, 432, 476, 438, 512, 423, 428, 499, 492, 
    493, 467, 493, 468, 420, 513, 427])

confidence_interval = st.t.interval(0.95, len(sample)-1, sample.mean(), sample.sem())

print("Cреднеео:", sample.mean())
print("95%-ый доверительный интервал:", confidence_interval)
print("p_value:", st.ttest_1samp(sample, confidence_interval[0]).pvalue.round(8))

Cреднеео: 470.5285714285714
95%-ый доверительный интервал: (463.3577536499037, 477.6993892072391)
p_value: 0.05


### Нахождение оценки стандартной ошибки с помощью bootstrap

In [6]:
pd.Series(sample.sample(frac=1, replace=True).mean() for _ in range(1000000)).std()

3.565998395920006

In [8]:
sample.sem()

3.5944932249097254

In [12]:
sample.std() / len(sample)**0.5

3.5944932249097254

### Bootstrap

In [30]:
import pandas as pd
import numpy as np
samples_A = pd.Series([
     98.24,  97.77,  95.56,  99.49, 101.4 , 105.35,  95.83,  93.02,
    101.37,  95.66,  98.34, 100.75, 104.93,  97.  ,  95.46, 100.03,
    102.34,  98.23,  97.05,  97.76,  98.63,  98.82,  99.51,  99.31,
     98.58,  96.84,  93.71, 101.38, 100.6 , 103.68, 104.78, 101.51,
    100.89, 102.27,  99.87,  94.83,  95.95, 105.2 ,  97.  ,  95.54,
     98.38,  99.81, 103.34, 101.14, 102.19,  94.77,  94.74,  99.56,
    102.  , 100.95, 102.19, 103.75, 103.65,  95.07, 103.53, 100.42,
     98.09,  94.86, 101.47, 103.07, 100.15, 100.32, 100.89, 101.23,
     95.95, 103.69, 100.09,  96.28,  96.11,  97.63,  99.45, 100.81,
    102.18,  94.92,  98.89, 101.48, 101.29,  94.43, 101.55,  95.85,
    100.16,  97.49, 105.17, 104.83, 101.9 , 100.56, 104.91,  94.17,
    103.48, 100.55, 102.66, 100.62,  96.93, 102.67, 101.27,  98.56,
    102.41, 100.69,  99.67, 100.99])
samples_B = pd.Series([
    101.67, 102.27,  97.01, 103.46, 100.76, 101.19,  99.11,  97.59,
    101.01, 101.45,  94.8 , 101.55,  96.38,  99.03, 102.83,  97.32,
     98.25,  97.17, 101.1 , 102.57, 104.59, 105.63,  98.93, 103.87,
     98.48, 101.14, 102.24,  98.55, 105.61, 100.06,  99.  , 102.53,
    101.56, 102.68, 103.26,  96.62,  99.48, 107.6 ,  99.87, 103.58,
    105.05, 105.69,  94.52,  99.51,  99.81,  99.44,  97.35, 102.97,
     99.77,  99.59, 102.12, 104.29,  98.31,  98.83,  96.83,  99.2 ,
     97.88, 102.34, 102.04,  99.88,  99.69, 103.43, 100.71,  92.71,
     99.99,  99.39,  99.19,  99.29, 100.34, 101.08, 100.29,  93.83,
    103.63,  98.88, 105.36, 101.82, 100.86, 100.75,  99.4 ,  95.37,
    107.96,  97.69, 102.17,  99.41,  98.97,  97.96,  98.31,  97.09,
    103.92, 100.98, 102.76,  98.24,  97.  ,  98.99, 103.54,  99.72,
    101.62, 100.62, 102.79, 104.19])

AB_difference = samples_B.mean() - samples_A.mean()
"Разность средних чеков:", AB_difference

('Разность средних чеков:', 0.7682000000000357)

#### Проверка нулевой гипотезы

In [137]:
alpha = 0.05
state = np.random.RandomState(12345)
bootstrap_samples = 1000
count = 0

for i in range(bootstrap_samples):
    united_samples = pd.concat([samples_A, samples_B])
    size = len(united_samples)//2
    subsample = united_samples.sample(frac=1, random_state=state, replace=True)
    subsample_A = subsample[:size]
    subsample_B = subsample[size:]
    bootstrap_difference = subsample_B.mean() - subsample_A.mean()
    if bootstrap_difference >= AB_difference:
        count += 1

pvalue = count / bootstrap_samples
print('p-value =', pvalue)
if pvalue < alpha:
    print("Отвергаем нулевую гипотезу: скорее всего, средний чек увеличился")
else:
    print("Не получилось отвергнуть нулевую гипотезу: скорее всего, средний чек не увеличился")

p-value = 0.034
Отвергаем нулевую гипотезу: скорее всего, средний чек увеличился


In [125]:
st.ttest_ind(samples_A, samples_B, alternative='less')

TtestResult(statistic=-1.8289566175344536, pvalue=0.034455195927867825, df=198.0)

#### То же самое сокращенно

In [270]:
united_samples = pd.concat([samples_A, samples_B])
size = len(united_samples)//2

boot_diffs = [sample[size:].mean() - sample[:size].mean() 
              for sample in (united_samples.sample(frac=1, replace=True)
              for _ in range(10000))
             ]
pvalue = np.mean(boot_diffs >= AB_difference)
pvalue

0.0347

### Boorstrap для моделей

In [738]:
target = pd.read_csv('datasets/eng_target.csv')['0']
probabilities = pd.read_csv('datasets/eng_probabilites.csv')['0']

def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    return target[probs_sorted.index][:count].sum() * 1000

state = np.random.RandomState(12345)

values = pd.Series([
    revenue(target[prob.index], prob, 10) 
    for prob in (
        probabilities.sample(25, random_state=state, replace=True) 
        for _ in range(1000)
    )
])

mean = values.mean()
lower = values.quantile(0.01)
print("Средняя выручка:", mean)
print("1%-квантиль:", lower)

Средняя выручка: 9167.0
1%-квантиль: 6000.0
